In [1]:
"""
This program tests Affine+Ditrortion35 model
on artificially generated data (pairs of star coordinates)
""";

In [2]:
from pylab import *
from PIL import Image, ImageDraw
from functools import partial

In [3]:
# for random consistency
np.random.seed(123456)

Create NUM_STAR_PAIRS (at least 5 because it's minimum needed for affine+distortion35 model)   
pairs of stars with coordinates in range from (MIN_X, MIN_Y) to (MAX_X, MAX_Y)
with OFFSET from edges

In [4]:
NUM_STAR_PAIRS = 20
N = NUM_STAR_PAIRS

MIN_X = 0
MAX_X = 2000
MIN_Y = 0
MAX_Y = 2000

OFFSET = 250

width = abs(MAX_X - MIN_X)
height = abs(MAX_Y - MIN_Y)

xCenter = (MAX_X - MIN_X) // 2
yCenter = (MAX_Y - MIN_Y) // 2


leftX = randint(low = MIN_X + OFFSET,
                high = MAX_X - OFFSET,
                size=NUM_STAR_PAIRS)
leftY = randint(low = MIN_Y + OFFSET,
                high = MAX_Y - OFFSET,
                size=NUM_STAR_PAIRS)



print('''\
Left X: {}
Left Y: {}
'''.format(leftX, leftY)
)

Left X: [ 315  996  748 1323  421 1001  805 1109  282  465  926  386  708 1540 1542
  782 1253  553  940  720]
Left Y: [1120 1160  960  444 1449  856 1191 1489 1486 1486 1703 1269 1215 1259 1438
 1733  669 1239  694 1116]



In [5]:
ELL_RAD = 3

# Draw star pairs
scatterOriginal = Image.new('RGB', (width, height), 'lightgray')

draw = ImageDraw.Draw(scatterOriginal)

# Central point
draw.ellipse((xCenter - ELL_RAD, yCenter - ELL_RAD, 
              xCenter + ELL_RAD, yCenter + ELL_RAD), fill='darkgreen')

for i in range(NUM_STAR_PAIRS): # draw star points
    draw.ellipse((leftX[i] - ELL_RAD, leftY[i] - ELL_RAD, 
                  leftX[i] + ELL_RAD, leftY[i] + ELL_RAD), fill='blue')


In [6]:
scatterOriginal.save('orig.png')

Create affine transformed pairs of stars coordinates  
with affine coeffincients  
(a,b,  
 c,d) -- for rotation matrix  
(e,f) -- for transition (shift)   

In [7]:
def affine_transform(xy, coeffs=(1,0,0,1,0,0)):
    assert coeffs != (1,0,0,1,0,0)
        
    _a, _b, _c, _d, _e, _f = coeffs
    x, y = xy
    return [
        _a * x + _b * y + _e,
        _c * x + _d * y + _f
    ]

In [8]:
A_COEFF = 0.9951
B_COEFF = 0.0070
C_COEFF = -0.0118
D_COEFF = 1.0023
E_COEFF = 241.96
F_COEFF = 50.92
AFFINE_COEFFS = (A_COEFF, B_COEFF, C_COEFF, D_COEFF, E_COEFF, F_COEFF)

affinePerfect = partial(affine_transform, coeffs=AFFINE_COEFFS)

rightCoords = array(list(map(affinePerfect, zip(leftX, leftY))))
rightX = rightCoords[:, 0]
rightY = rightCoords[:, 1]

print('''\
Right X: {}
Right Y: {}
'''.format(rightX, rightY)
)

Right X: [  563.2565  1241.1996   993.0148  1561.5853   671.0401  1244.0471
  1051.3525  1355.9489   532.9802   715.0835  1175.3436   634.9516
   954.9958  1783.227   1786.4702  1032.2592  1493.5033   800.9233
  1182.212    966.244 ]
Right Y: [ 1169.779   1201.8352  1004.3016   480.3298  1498.2849   897.077
  1235.1603  1530.2585  1537.0102  1534.8508  1746.9101  1318.2839
  1260.3601  1294.6437  1474.0318  1778.6783   706.6733  1286.2443
   735.4242  1160.9908]



Create distorted pairs of stars coordinates  
with distortion coefficients  
eps1, eps2 -- for 3rd order distortion  
eps3, eps4 -- for 5th order distortion  

In [9]:
def distort_transform(xy, coeffs=(0,0)):
    assert coeffs != (0,0)
    
    # eps1, eps3 -- for left img
    # eps2, eps4 -- for right img
    _eps1_or_eps2, _eps3_or_eps4  = coeffs
    
    x, y = xy
    
    # squared distance from center to (x, y) point
    _r = (x - xCenter) ** 2 + (y - yCenter) ** 2
    
    return [
        x + (x - xCenter) * ( _r * _eps1_or_eps2 + (_r ** 2) * _eps3_or_eps4 ),
        y + (y - yCenter) * ( _r * _eps1_or_eps2 + (_r ** 2) * _eps3_or_eps4 )
    ]

In [10]:
EPS1 = 3e-8
EPS2 = 4e-8
EPS3 = 2e-14
EPS4 = -2e-14

DISTORT_COEFFS_LEFT = (EPS1, EPS3)
DISTORT_COEFFS_RIGHT = (EPS2, EPS4)

distortLeft = partial(distort_transform, coeffs=DISTORT_COEFFS_LEFT)
distortRight = partial(distort_transform, coeffs=DISTORT_COEFFS_RIGHT)

leftCoords = array(list(map(distortLeft, zip(leftX, leftY))))
leftX = leftCoords[:, 0]
leftY = leftCoords[:, 1]

rightCoords = array(list(map(distortRight, zip(rightX, rightY))))
rightX = rightCoords[:, 0]
rightY = rightCoords[:, 1]


print('''\
Left X: {}
Left Y: {}
'''.format(leftX, leftY)
)
print('''\
Right X: {}
Right Y: {}
'''.format(rightX, rightY)
)

Left X: [  301.85717022   995.99687359   747.48645156  1328.11083421   408.33770618
  1001.00063071   804.54249044  1109.95812091   257.69335992   453.69485935
   924.52116535   375.24324542   706.74718651  1547.20007445  1552.45222209
   776.68417575  1254.46983662   548.96441226   939.81362939   719.18115946]
Left Y: [ 1122.30239354  1160.12505657   959.91848438   435.20240303  1458.81929175
   855.90917769  1191.44812475  1493.29835894  1502.45268396  1496.26971656
  1717.04892915  1273.71264981  1215.92244829  1262.45336904  1446.44662966
  1750.87384942   667.07701217  1241.15773036   693.0495099   1116.33923394]

Right X: [  559.84178869  1242.10672889   993.0147812   1570.88668006   667.1852761
  1244.70789628  1051.46806176  1360.57186778   525.91470077   711.66667698
  1178.25699198   631.92820181   954.87451215  1797.48416878  1801.81308189
  1032.80492887  1498.93721696   800.01409263  1182.92536935   966.20796014]
Right Y: [ 1171.10642965  1202.59428311  1004.30161158   471

In [11]:
# inputLeftX, inputLeftY, inputRightX, inputRightY are coordinates we get from measuring system
inputLeftX = leftX
inputLeftY = leftY

inputRightX = rightX
inputRightY = rightY

In [12]:
def correct_distort(xy, coeffs=(0,0)):
    assert coeffs != (0,0)
    
    # eps1, eps3 -- for left img
    # eps2, eps4 -- for right img
    _eps1_or_eps2, _eps3_or_eps4  = coeffs
    
    x, y = xy
    
    # squared distance from center to (x, y) point
    _r = (x - xCenter) ** 2 + (y - yCenter) ** 2
    
    return [
        x - (x - xCenter) * ( _r * _eps1_or_eps2 + (_r ** 2) * _eps3_or_eps4 ),
        y - (y - yCenter) * ( _r * _eps1_or_eps2 + (_r ** 2) * _eps3_or_eps4 )
    ]

Check out that if we do backward distort+affine transform then coordinates fit well.

In [13]:
# Backward distort

correctDistortLeftPerfect = partial(correct_distort, coeffs=DISTORT_COEFFS_LEFT)
leftCoords = array(list(map(correctDistortLeftPerfect, zip(leftX, leftY))))
leftX = leftCoords[:, 0]
leftY = leftCoords[:, 1]


correctDistortRightPerfect = partial(correct_distort, coeffs=DISTORT_COEFFS_RIGHT)
rightCoords = array(list(map(correctDistortRightPerfect, zip(rightX, rightY))))
rightX = rightCoords[:, 0]
rightY = rightCoords[:, 1]


# Backward affine
leftCoords = array(list(map(affinePerfect, zip(leftX, leftY))))
leftX = leftCoords[:, 0]
leftY = leftCoords[:, 1]

print('IF PERFECTLY FOUND COEFFICIENTS')
print('''Backward distort+affine Left:
Left X: {}
Left Y: {}
'''.format(leftX, leftY)
)

print('''Backward distort Right:
Right X: {}
Right Y: {}
'''.format(rightX, rightY)
)

IF PERFECTLY FOUND COEFFICIENTS
Backward distort+affine Left:
Left X: [  564.15350127  1241.19960531   993.01802151  1561.30671411   672.03621066
  1244.04710002  1051.35579107  1355.92018012   536.12012721   715.93848164
  1175.44228552   635.61392005   955.01270254  1782.8968358   1785.74780543
  1032.72317631  1493.47611337   801.04354     1182.21386921   966.25142578]
Left Y: [ 1169.60987897  1201.83490248  1004.30207632   480.82208144  1497.49070769
   897.07717394  1235.1569913   1530.13302392  1534.82181389  1534.05327067
  1745.89692614  1317.9828493   1260.34729785  1294.48863598  1473.4556375
  1777.0632429    706.70978378  1286.17786977   735.43344798  1160.98760398]

Backward distort Right:
Right X: [  563.3304553   1241.1896863    993.0148      1561.25050152   671.15679627
  1244.04184914  1051.35173371  1355.79861007   533.2293412    715.18868907
  1175.23872803   635.02049598   954.9967594   1782.73205251  1786.02155487
  1032.23957284  1493.346353     800.93526825  1182

In [14]:
delX = abs(leftX - rightX)
delY = abs(leftY - rightY)
print("delX:", delX)
print("delY:", delY)

sigSqr = 1.0 / N * sum(delX**2 + delY**2)
mX = max(delX)
mY = max(delY)
m = max(mX, mY)

print("IF PERFECTLY FOUND COEFFICIENTS")
print("mX: %.4f mY: %.4f m: %.4f" % (mX, mY, m))
print("sigSqr: %.4f" % sigSqr)

delX: [ 0.82304597  0.00991901  0.00322151  0.0562126   0.87941439  0.00525088
  0.00405735  0.12157005  2.89078602  0.74979257  0.2035575   0.59342407
  0.01594314  0.16478329  0.27374944  0.48360347  0.12976037  0.10827176
  0.00997259  0.00731128]
delY: [  1.40371754e-01   7.99823767e-03   4.76317475e-04   1.82471342e-01
   6.17429162e-01   2.04053308e-03   2.00375038e-04   9.84114224e-02
   1.90190711e+00   6.00066364e-01   5.66451342e-01   2.40980641e-01
   7.25179103e-03   3.11312483e-02   3.05749128e-01   1.14129316e+00
   5.68018081e-02   4.92215786e-02   2.51830987e-03   2.64994996e-03]
IF PERFECTLY FOUND COEFFICIENTS
mX: 2.8908 mY: 1.9019 m: 2.8908
sigSqr: 0.8675


### Test affine model

Calculate model coefficients

In [15]:
leftX = inputLeftX
leftY = inputLeftY
rightX = inputRightX
rightY = inputRightY

In [16]:
xi = np.zeros(2 * NUM_STAR_PAIRS)

for i in range(NUM_STAR_PAIRS): # fill the xi vector
    xi[2 * i] = rightX[i]
    xi[2 * i + 1] = rightY[i]

print('xi:\n', xi)

xi:
 [  559.84178869  1171.10642965  1242.10672889  1202.59428311   993.0147812
  1004.30161158  1570.88668006   471.722648     667.1852761   1504.12391121
  1244.70789628   896.79831963  1051.46806176  1235.68949601  1360.57186778
  1537.1453536    525.91470077  1545.1345775    711.66667698  1541.26492669
  1178.25699198  1759.32025865   631.92820181  1320.91998597   954.87451215
  1261.06177931  1797.48416878  1300.00713226  1801.81308189  1483.27946624
  1032.80492887  1791.85120031  1498.93721696   703.44350816   800.01409263
  1287.55161234  1182.92536935   734.38837214   966.20796014  1161.16268312]


In [17]:
k = 6 # num of coeff-s

z = np.zeros(k)
arr = np.zeros((2 * NUM_STAR_PAIRS, k)) # matrix A

for i in range(NUM_STAR_PAIRS): # fill the A matrix
    
    arr[2 * i] = [leftX[i], leftY[i], 0, 0, 1, 0]

    arr[2 * i + 1] = [0, 0, leftX[i], leftY[i], 0, 1]

    
p_arr = pinv(arr, rcond=1e-20)
z = np.dot(p_arr, xi)

print("""
Affine coefficients:
%.4f %.4f %.4f %.4f 
%.2f %.2f""" % tuple(z))
print("""
Perfect Affine coefficients:
%.4f %.4f %.4f %.4f 
%.2f %.2f""" % AFFINE_COEFFS)
print('cond(A): ', np.linalg.cond(arr))


Affine coefficients:
0.9916 0.0107 -0.0078 1.0007 
245.11 48.30

Perfect Affine coefficients:
0.9951 0.0070 -0.0118 1.0023 
241.96 50.92
cond(A):  7553.54281032


Calculate error metrics

In [18]:
"""
Align images and blend

a) Affine
""";

In [19]:
affine = partial(affine_transform, coeffs=tuple(z))

# Calc estimated (affine transformed) points
leftCoords = array(list(map(affine, zip(leftX, leftY))))

# Estimated coordinates
estLeftX = leftCoords[:, 0]
estLeftY = leftCoords[:, 1]


print('''Backward affine Left:
Left X: {}
Left Y: {}
'''.format(estLeftX, estLeftY)
)

print('''Right:
Right X: {}
Right Y: {}
'''.format(rightX, rightY)
)

Backward affine Left:
Left X: [  556.42121457  1245.14446771   996.5795505   1566.73121792   665.60301507
  1246.8571424   1055.62998327  1361.70854259   516.68785088   710.97982826
  1180.21621818   630.80905383   958.91609345  1792.81869219  1799.99190872
  1033.98011307  1496.18420215   802.72585961  1184.44380347   970.18222522]
Left Y: [ 1169.05161982  1201.51948329  1003.09792171   473.50909198  1504.98108806
   897.04956758  1234.3490306   1534.04499315  1549.81327533  1542.1063632
  1759.39153952  1320.00005301  1259.59886283  1299.64680136  1483.72952458
  1794.38655      706.11868812  1286.0752658    734.54904443  1159.84879113]

Right:
Right X: [  559.84178869  1242.10672889   993.0147812   1570.88668006   667.1852761
  1244.70789628  1051.46806176  1360.57186778   525.91470077   711.66667698
  1178.25699198   631.92820181   954.87451215  1797.48416878  1801.81308189
  1032.80492887  1498.93721696   800.01409263  1182.92536935   966.20796014]
Right Y: [ 1171.10642965  1202.5

1) $\Delta x_i, \Delta y_i, \; i = 1,N$

2) $\sigma^2 = \frac{1}{N} \sum\limits_{i=1}^{N} 
                \left( \Delta x_i^2 + \Delta y_i^2 \right)$

In [20]:
delX = abs(estLeftX - rightX)
delY = abs(estLeftY - rightY)
print("delX:", delX)
print("delY:", delY)

sigSqr = 1.0 / N * sum(delX**2 + delY**2)
mX = max(delX)
mY = max(delY)
m = max(mX, mY)

print("mX: %.4f mY: %.4f m: %.4f" % (mX, mY, m))
print("sigSqr: %.4f" % sigSqr)

delX: [ 3.42057412  3.03773881  3.56476931  4.15546215  1.58226102  2.14924612
  4.16192151  1.13667481  9.22684989  0.68684871  1.9592262   1.11914798
  4.0415813   4.6654766   1.82117318  1.17518421  2.75301481  2.71176698
  1.51843413  3.97426509]
delY: [ 2.05480983  1.07479981  1.20368987  1.78644398  0.85717685  0.25124795
  1.34046541  3.10036045  4.67869783  0.8414365   0.07128086  0.91993296
  1.46291649  0.3603309   0.45005834  2.5353497   2.67517996  1.47634654
  0.16067229  1.313892  ]
mX: 9.2268 mY: 4.6787 m: 9.2268
sigSqr: 15.4416


Plot aligned star pairs

In [21]:
scatter = Image.new('RGB', (width, height), 'lightgray')


draw = ImageDraw.Draw(scatter)
draw.ellipse((xCenter - ELL_RAD, yCenter - ELL_RAD, 
              xCenter + ELL_RAD, yCenter + ELL_RAD), fill='darkgreen')


for i in range(NUM_STAR_PAIRS): # draw star points
    draw.ellipse((estLeftX[i] - ELL_RAD, estLeftY[i] - ELL_RAD, 
                  estLeftX[i] + ELL_RAD, estLeftY[i] + ELL_RAD), fill='red')
    
    draw.ellipse((rightX[i] - ELL_RAD, rightY[i] - ELL_RAD, 
                  rightX[i] + ELL_RAD, rightY[i] + ELL_RAD), fill='blue')

In [22]:
scatter.save('000.png')

### Test affine+distortion3 model

Calculate model coefficients

In [23]:
leftX = inputLeftX
leftY = inputLeftY
rightX = inputRightX
rightY = inputRightY

Calculate error metrics

Plot aligned star pairs

### Test affine+distortion35 model

Calculate model coefficients

In [24]:
leftX = inputLeftX
leftY = inputLeftY
rightX = inputRightX
rightY = inputRightY

In [25]:
"""
c) Affine + Ditortion 3rd, 5th orders 
  (at least 5 stars)
""";

In [26]:
k35 = 10

z35 = np.zeros(k35)
arr35 = np.zeros((2 * N, k35)) # matrix A

for i in range(N): # fill the A matrix
    dist_l = (leftX[i] - xCenter) ** 2 + (leftY[i] - yCenter) ** 2
    dist_r = (rightX[i] - xCenter) ** 2 + (rightY[i] - yCenter) ** 2

    zx1 = (leftX[i] - xCenter) * dist_l
    zx2 = (rightX[i] - xCenter) * dist_r
    wx1 = (leftX[i] - xCenter) * dist_l ** 2
    wx2 = (rightX[i] - xCenter) * dist_r ** 2

    arr35[2 * i] = [leftX[i], leftY[i], 0, 0, 1, 0, -zx1, zx2, -wx1, wx2]

    zy1 = (leftY[i] - yCenter) * dist_l
    zy2 = (rightY[i] - yCenter) * dist_r
    wy1 = (leftY[i] - yCenter) * dist_l ** 2
    wy2 = (rightY[i] - yCenter) * dist_r ** 2

    arr35[2 * i + 1] = [0, 0, leftX[i], leftY[i], 0, 1, -zy1, zy2, -wy1, wy2]


In [27]:
p_arr35 = pinv(arr35, rcond=1e-20)
z35 = np.dot(p_arr35, xi)


print("""
Affine coefficients + Ditortion 3rd, 5th orders:

%.4f %.4f %.4f %.4f 
%.2f %.2f 
%.2e %.2e 
%.2e %.2e""" % tuple(z35))

print("""
Perfect Affine coefficients:
%.4f %.4f %.4f %.4f 
%.2f %.2f
%.2e %.2e
%.2e %.2e""" % tuple( list(AFFINE_COEFFS) + [EPS1, EPS2, EPS3, EPS4] ) )


print('cond(A): ', np.linalg.cond(arr35))


Affine coefficients + Ditortion 3rd, 5th orders:

0.9957 0.0069 -0.0116 1.0027 
241.57 50.32 
3.24e-08 3.77e-08 
1.07e-14 -1.84e-14

Perfect Affine coefficients:
0.9951 0.0070 -0.0118 1.0023 
241.96 50.92
3.00e-08 4.00e-08
2.00e-14 -2.00e-14
cond(A):  3.65549856286e+15


Calculate error metrics

In [29]:
"""
c) Affine + Ditortion3,5
""";

In [30]:
a = float(z35[0])
b = float(z35[1])
c = float(z35[2])
d = float(z35[3])
e = float(z35[4])
f = float(z35[5])

eps1 = float(z35[6])
eps2 = float(z35[7])
eps3 = float(z35[8])
eps4 = float(z35[9])

In [31]:
# Backward distort

correctDistortLeft = partial(correct_distort, coeffs=(eps1, eps3))
leftCoords = array(list(map(correctDistortLeft, zip(leftX, leftY))))
leftX = leftCoords[:, 0]
leftY = leftCoords[:, 1]


correctDistortRight = partial(correct_distort, coeffs=(eps2, eps4))
rightCoords = array(list(map(correctDistortRight, zip(rightX, rightY))))
estRightX35 = rightCoords[:, 0]
estRightY35 = rightCoords[:, 1]


# Backward affine
affine = partial(affine_transform, coeffs=(a,b,c,d,e,f))


leftCoords = array(list(map(affine, zip(leftX, leftY))))
estLeftX35 = leftCoords[:, 0]
estLeftY35 = leftCoords[:, 1]


print('''Backward distort+affine Left:
Left X: {}
Left Y: {}
'''.format(estLeftX35, estLeftY35)
)

print('''Backward distort Right:
Right X: {}
Right Y: {}
'''.format(estRightX35, estRightY35)
)

Backward distort+affine Left:
Left X: [  563.02224792  1241.2490874    992.9782806   1561.83700809   670.79757653
  1244.13217092  1051.31839914  1355.99746916   532.71560271   714.86074016
  1175.30367128   634.74476744   954.93788145  1783.45300831  1786.72141846
  1032.14561386  1493.69055682   800.82834938  1182.29101239   966.18905162]
Left Y: [ 1169.66306459  1201.89581399  1004.24410084   480.30378331  1498.28588084
   897.0381323   1235.17137046  1530.35325529  1536.95864834  1534.86811418
  1747.079286    1318.21368881  1260.3401805   1294.81675619  1474.26933528
  1778.85042056   706.69324048  1286.18219522   735.35646337  1160.96190566]

Backward distort Right:
Right X: [  563.13526009  1241.24241288   993.0147989   1561.72685447   670.9427881
  1244.0803682   1051.35847743  1356.05163315   532.85528783   714.99953118
  1175.38771172   634.84812385   954.98968908  1783.41882595  1786.68650036
  1032.26723171  1493.65005749   800.88255544  1182.2453418    966.24200617]
Right 

In [32]:
delX35 = abs(estLeftX35 - estRightX35)
delY35 = abs(estLeftY35 - estRightY35)
print("delX35:", delX35)
print("delY35:", delY35)

sigSqr35 = 1.0 / N * sum(delX35**2 + delY35**2)
mX35 = max(delX35)
mY35 = max(delY35)
m35 = max(mX35, mY35)

print("mX35: %.4f mY35: %.4f m35: %.4f" % (mX35, mY35, m35))
print("sigSqr35: %.4f" % sigSqr35)

delX35: [ 0.11301217  0.00667452  0.03651829  0.11015362  0.14521157  0.05180273
  0.0400783   0.05416399  0.13968512  0.13879102  0.08404044  0.10335641
  0.05180763  0.03418236  0.0349181   0.12161785  0.04049933  0.05420606
  0.04567059  0.05295454]
delY35: [ 0.16306603  0.02478829  0.05749984  0.10497257  0.14642026  0.02483736
  0.01630221  0.05828669  0.19518392  0.14031373  0.01871645  0.16043154
  0.05527264  0.10089281  0.10716385  0.02175021  0.10716966  0.12068972
  0.0193236   0.03840344]
mX35: 0.1452 mY35: 0.1952 m35: 0.1952
sigSqr35: 0.0172


Plot aligned star pairs

In [33]:
scatter35 = Image.new('RGB', (width, height), 'lightgray')


draw = ImageDraw.Draw(scatter35)
draw.ellipse((xCenter - ELL_RAD, yCenter - ELL_RAD, 
              xCenter + ELL_RAD, yCenter + ELL_RAD), fill='darkgreen')


for i in range(NUM_STAR_PAIRS): # draw star points
    draw.ellipse((estLeftX35[i] - ELL_RAD, estLeftY35[i] - ELL_RAD, 
                  estLeftX35[i] + ELL_RAD, estLeftY35[i] + ELL_RAD), fill='red')
    
    draw.ellipse((estRightX35[i] - ELL_RAD, estRightY35[i] - ELL_RAD, 
                  estRightX35[i] + ELL_RAD, estRightY35[i] + ELL_RAD), fill='blue')


In [34]:
scatter35.save('035.png')